# ML Assigment

Vuyo Ncume 2095458 <br/>
Vhugala Mudau 2094007 <br/>
Amatullah Mthethwa 1870175 <br/>
Melissa Mmatshaka 2111308 <br/>
Rumbidzai Moyo 1856080

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix

### The data:

In [2]:
d = pd.read_csv('heart_failure_clinical_records_dataset.csv')

# dropping missing values if there are any
data = d.dropna().copy()

data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


### Splitting into training, validation and testing data

Note: 60% of the data is for training, 20% for validation and the rest for testing.

In [3]:
training_data = data.loc[0:179,]
training_data.tail()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
175,60.0,1,95,0,60,0,337000.0,1.0,138,1,1,146,0
176,69.0,0,1419,0,40,0,105000.0,1.0,135,1,1,147,0
177,49.0,1,69,0,50,0,132000.0,1.0,140,0,0,147,0
178,63.0,1,122,1,60,0,267000.0,1.2,145,1,0,147,0
179,55.0,0,835,0,40,0,279000.0,0.7,140,1,1,147,0


In [4]:
validation_data = data.loc[180:239,]
validation_data.tail()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
235,77.0,1,109,0,50,1,406000.00,1.10,137,1,0,209,0
236,75.0,0,119,0,50,1,248000.00,1.10,148,1,0,209,0
237,70.0,0,232,0,30,0,173000.00,1.20,132,1,0,210,0
238,65.0,1,720,1,40,0,257000.00,1.00,136,0,0,210,0
239,55.0,1,180,0,45,0,263358.03,1.18,137,1,1,211,0


In [5]:
testing_data = data.loc[240:298,]
testing_data.tail()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
294,62.0,0,61,1,38,1,155000.0,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.0,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.0,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.0,1.4,140,1,1,280,0
298,50.0,0,196,0,45,0,395000.0,1.6,136,1,1,285,0


## Neural Network

Note: we will use the same features as in the ID3 algorithm so we can compare which solution is best for this set of features.

In [17]:
# Defining the X and Y matrices
Y = training_data['DEATH_EVENT'].values
X = training_data[['age', 'anaemia', 'diabetes', 'high_blood_pressure', 'smoking']]

# validation and test data
Y_validate = validation_data['DEATH_EVENT'].values
X_validate = validation_data[['age', 'anaemia', 'diabetes', 'high_blood_pressure', 'smoking']]
Y_test = testing_data['DEATH_EVENT'].values
X_test = testing_data[['age', 'anaemia', 'diabetes', 'high_blood_pressure', 'smoking']]

# Saving the feature list 
features = list(X.columns)

In [ ]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs, deepness):
    network = list()
    for i in range(deepness):
        hidden_layer = [{'weights':[np.random.rand() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
        network.append(hidden_layer)
    output_layer = [{'weights':[np.random.rand() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

# sum inputs
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

# sigmoid
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

# sigmoid derivative
def transfer_derivative(output):
    return output * (1.0 - output)

# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs
 
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
            
# update weights
def update_weights(network, row, l_rate, out):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']
            out = np.append(out, l_rate * neuron['delta'])
    return out
            
# train network
def train_network(network, train, l_rate, n_epoch, n_outputs, e):
    t = np.array([])
    c = 0
    prev_network = np.array([45,45,45])
    while(sum(abs( np.array(network[0][0]['weights'])-prev_network)) > e and c<n_epoch):
        prev_network = network[0][0]['weights']
        for row in train:
            outputs = forward_propagate(network, row)
            backward_propagate_error(network, row[-1])
            update_weights(network, row, l_rate, t)
        c += 1
#     for epoch in range(n_epoch):
#         for row in train:
#             outputs = forward_propagate(network, row)
#             backward_propagate_error(network, row[-1])
#             update_weights(network, row, l_rate, t)
        
# Make a prediction with a network
def predict(network, row):
    ans = np.array([])
    outputs = forward_propagate(network, row)
    for i in outputs:
        if i > 0.5:
            ans = np.append(ans, 1)
        else:
            ans = np.append(ans, 0)
    return ans

# calls predict() to make predictions
def make_prediction(network, data):
    prediction = []
    y = []
    for i in data:
        y.append(i[len(i)-1])
    for i in range(len(data)):
        row = data[i]
        prediction.append(predict(network, row))
    
    result = confusion_matrix(y, prediction)
    print('Confusion matrix: \n\n', result)
    print('\n\n\nAccuracy: ', (result[0,0]+result[1,1])/len(y)*100,'%')

# splitting up data
# data = df[['x1', 'x2', 'class']].to_numpy()
# training = data[0:60]
# validation = data[60:80]
# testing = data[80:100]